In [ ]:
import tensorflow.compat.v1 as tf
import numpy as np
tf.disable_v2_behavior()

xy = np.loadtxt('data-01.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, :-1]
y_data = xy[:, [-1]]

print('x_data:\n', x_data.shape, '\n', x_data, '\n')
print('y_data:\n', y_data.shape, '\n', y_data, '\n')

X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([3, 1]), name = 'weight')
b = tf.Variable(tf.random_normal([1]), name = 'bias')

hypothesis = tf.matmul(X,W) + b

cost = tf.reduce_mean(tf.square(hypothesis - Y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(2001):
    cost_val, hy_val, _ = sess.run([cost, hypothesis, train], 
                                  feed_dict = {X: x_data, Y: y_data})
    if step % 10 == 0:
        print(step, "Cost: ",cost_val, "\nPrediction:\n", hy_val)

#Prediction with new data
print("Your score will be ", sess.run(hypothesis, feed_dict={X: [[100,70,101]]}))
print("Other scores will be ", sess.run(hypothesis, feed_dict={X: [[60,70,110],[90,100,80]]}))


In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
filename_queue = tf.train.string_input_producer(['data-01.csv'], shuffle = False, name = 'filename_queue')

reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

#Default values, in case of empty columns. Also specifies the type of the decoded result.
record_defaults = [[0.],[0.],[0.],[0.]]
xy = tf.decode_csv(value, record_defaults = record_defaults)
print(xy)

#collect batches of csv in 
train_x_batch, train_y_batch = tf.train.batch([xy[0:-1], xy[-1:]], batch_size=10)

X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([3, 1]), name = 'weight')
b = tf.Variable(tf.random_normal([1]), name = 'bias')

hypothesis = tf.matmul(X,W) + b

cost = tf.reduce_mean(tf.square(hypothesis - Y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

#Start population the filename queue (통상적으로 쓰는 코드이니 넘어가자)
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

for step in range(2001):
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    cost_val, hy_val, _ = sess.run([cost, hypothesis, train],
                                  feed_dcit={X: x_batch, Y:y_batch})
    if step % 10 == 0:
        print(step, "Cost: ", cost_val, "\nPrediction:\n", hy_val)
        
#이것도 통상적으로 쓰는 코드
coord.request_stop()
coord.join(threads)